<a href="https://colab.research.google.com/github/FelipeBZanardo/Desafio_Projeto_ETL_DIO/blob/main/ETL_MinhaQuina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** O objetivo desse projeto é gerar uma lista com 5 dezenas para jogar na Quina (Caixa Federal) através de uma API desenvolvida com JAVA disponível em https://github.com/FelipeBZanardo/Projeto-Final-Web-II.

**Condições do Problema:**

1. Extrair os dados de todos os sorteios já realizados pela caixa através do endpoint https://servicebus2.caixa.gov.br/portaldeloterias/api/quina/
2. Com base nos dados do item anterior, foi levantado as 10 dezenas que mais saíram, e através de um sorteio randômico, foram geradas 5 dezenas para realização da aposta.
3. Por fim, uma aposta é realizada através API citada no início.



## **E**xtract

Extraia os dados de todos os sorteios já realizados na caixa!
Como atualmente são mais de 6000 sorteios, para acelerar o processo, faça o download do arquivo.csv 'banco_dados.csv' disponível em: '' e faça o upload na pasta aqui do colab.
Não se preocupe, os sorteios mais recentes serão atualizados pelo programa descrito abaixo.

In [92]:
import os.path
import pandas as pd
from requests.models import Response
import requests
import json
import warnings

warnings.filterwarnings("ignore") #ignora warnings

end_point_caixa = 'https://servicebus2.caixa.gov.br/portaldeloterias/api/quina/'
nome_arquivo = 'banco_dados.csv'

#Verifica se foi feito o upload do arquivo 'banco_dados.csv'
def get_banco_dados_df():
  if not os.path.isfile(nome_arquivo):
    pd.DataFrame().to_csv(nome_arquivo)
  return pd.read_csv(nome_arquivo)

#atualiza o arquivo com todos os sorteios
def atualizar_banco_dados():
  banco_dados_df = get_banco_dados_df()
  novos_sorteios_df = pd.DataFrame(get_sorteios(get_ultimo_sorteio_salvo()))
  if banco_dados_df.empty:
    novos_sorteios_df.to_csv(nome_arquivo)
  else:
    pd.concat([banco_dados_df, novos_sorteios_df]).drop(columns='Unnamed: 0').to_csv(nome_arquivo)

#verifica qual o último sorteio salvo no arquivo de banco de dados
def get_ultimo_sorteio_salvo():
  banco_dados_df = get_banco_dados_df()
  if banco_dados_df.empty:
    return 0
  return banco_dados_df['numero_sorteio'].iloc[-1]

#Busca na API da Caixa o último sorteio
def get_ultimo_sorteio_quina():
  return requests.get(end_point_caixa, verify=False).json()

#todos os sorteios a partir do numero_sorteio_inicial
def get_sorteios(numero_sorteio_inicial):
  numeros_sorteio = []
  datas_sorteio = []
  dezenas_sorteios = []
  ultimo_sorteio = get_ultimo_sorteio_quina()['numero']
  for numero_sorteio in range(numero_sorteio_inicial + 1, ultimo_sorteio + 1, 1):
    response = requests.get(f'{end_point_caixa}{numero_sorteio}', verify=False).json()
    numeros_sorteio.append(response['numero'])
    datas_sorteio.append(response['dataApuracao'])
    dezenas_sorteios.append(response['listaDezenas'])

  return {
      'numero_sorteio': numeros_sorteio,
      'data_sorteio': datas_sorteio,
      'dezenas_sorteadas': dezenas_sorteios
      }

atualizar_banco_dados()


## **T**ransform

Feito um levantamento das 10 dezenas mais sorteadas dos 'n' últimos sorteios e através de uma escolha randômica foi escolhido 5 números para a realização da aposta.

In [73]:
from ast import literal_eval
import random

#coloque aqui quantos sorteios você quer filtrar.
#n = 0 será pego todos os sorteios
n = 0

n = get_ultimo_sorteio_quina()['numero'] if n == 0 else n

#Série com a lista de dezenas sorteadas de cada sorteio
dezenas = get_banco_dados_df()['dezenas_sorteadas'].apply(literal_eval).head(n)

#As 10 dezenas mais sorteadas pela Caixa
dezenas_explode = dezenas.explode().reset_index(drop=True).dropna()
lista_dezenas = dezenas_explode.groupby(dezenas_explode).count().sort_values(ascending=False).head(10).index.to_list()

#Sugestão de dezenas a serem apostadas
dezenas_a_apostar = random.sample(lista_dezenas, k=5)
print(dezenas_a_apostar)


['32', '54', '52', '39', '38']


## **L**oad

Aposta feita com as dezenas sugeridas pelo algoritmo através da API 'Minha-Quina' https://github.com/FelipeBZanardo/Projeto-Final-Web-II

## **I**mportante

Para realização dos próximos passos, seguir as seguintes instruções:

1.   Fazer o clone da API  https://github.com/FelipeBZanardo/Projeto-Final-Web-II
2.   Abrir o IntelliJ e rodar a aplicação 'MinhaQuinaAplication'
3.  Para gerar uma URL de um servidor local disponível aqui no Google Colab:
  *   Abrir o Terminal (windows + cmd)
  *   Digitar: `ssh -p 443 -R0:localhost:8080 -L4300:localhost:4300 qr@a.pinggy.io`
  * Copiar a url que aparecerá e substituir no código abaixo como indicado





In [76]:
#Login para utilização da API

#Substitua aqui a URL vinda do Terminal
url = 'http://rnbns-177-148-143-42.a.free.pinggy.online'

login = {'username': 'felipe.zanardo',
         'password': 'D7e-kx;$7wQD5e'}

token = requests.post(url=url+'/auth/login', json=login).json()['token']

In [78]:
#Realização da Aposta:

def formatar_data():
  data = get_ultimo_sorteio_quina()['dataProximoConcurso'].split('/')
  return data[2] + '-' + data[1] + '-' + data[0]

aposta = {'numeroSorteio': get_ultimo_sorteio_quina()['numero'] + 1,
	'dezenas': dezenas_a_apostar,
	'dataJogo': formatar_data()}

headers = {
    'Authorization': f'Bearer {token}',
}

response = requests.post(url=url+'/minha-quina/api/v1/apostas', json=aposta, headers=headers).json()
print(response)



{'id': 2, 'numeroSorteio': 6228, 'dezenas': [32, 54, 52, 39, 38], 'dataJogo': '2023-08-29'}
